In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import io
from PIL import Image
import hashlib

In [2]:
#!pip install selenium

In [3]:
import selenium
from selenium import webdriver
DRIVER_PATH = r'C:\Users\vivek\Downloads\chromedriver_win32\chromedriver.exe'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [4]:
wd.get('https://google.com')
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('indian terrorist photos')

In [5]:

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [6]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [7]:
def search_and_download(search_term:str,driver_path:str,target_path=r'C:\Users\vivek\Desktop\New folder\images\terrorist',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [8]:
search_term='indian terrorist photos'
search_and_download(
    search_term=search_term,
    driver_path=DRIVER_PATH
)

Found: 100 search results. Extracting links from 0:100
Found: 100 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTtpNH_Nk1yJLU-Yfd6pWn09rua33EfABSVsg&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\547360b4fb.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcR7S3fW1_Nxk8rJxApYPo1M35cTf3HnvTaMRA&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\aff2d463f4.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQMzzTaoIuZdtDypOsf-EvXvmBxAety7AfK7Q&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\f5f380ec5d.jpg
SUCCESS - saved https://images.financialexpress.com/2016/09/all-pakistani-terro-chief-L.jpg - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\92dba82e51.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQkH8A6P5aYLNK6J31

C:\ProgramData\Anaconda3\lib\site-packages\PIL\Image.py:2685: UserWarning: image file could not be identified because WEBP support not installed
  warnings.warn(message)


ERROR - Could not save https://qph.fs.quoracdn.net/main-qimg-ff79583e9284ab818fc8927607edf1ec.webp - cannot identify image file <_io.BytesIO object at 0x000002AD03FC4570>
SUCCESS - saved https://im.rediff.com/news/2015/nov/19terror.jpg - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\e0d0649b03.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRhMKeXJ2l2Laf1i8srxzxjrZxqK0kBT3vzmQ&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\e2a2e8ac5f.jpg
SUCCESS - saved https://assets.bwbx.io/images/users/iqjWHBFdfxIU/iay90FpYRU2g/v1/1000x-1.jpg - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\83ca6b1782.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQW4UVKEOKgkHdI9VV4gOjHdxj2oQRII1jDIQ&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\a37be6f085.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/imag

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQDvVM3ZR9ZoECAu1k-25WfA0toTxZ-TtQzxA&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\1b5285f240.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQAV7MvrmYZe7QkeR-WkmNXecM3c3qsNWhB-w&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\9689ef0bfb.jpg
SUCCESS - saved https://www.irishtimes.com/polopoly_fs/1.3971921.1564493919!/image/image.jpg_gen/derivatives/box_620_330/image.jpg - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\17d84c0cd6.jpg
SUCCESS - saved https://images.financialexpress.com/2017/03/Afzal-guru.jpg - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\8a1b653f79.jpg
SUCCESS - saved https://m.jagranjosh.com/imported/images/E/GK/us_blocks_pakistans_proposal_to_designate_indian_citizen_terrorist.jpg - as C:\Users\vivek\Desktop\New folder\images

SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcSjdPpc7m_8G3aADRP5uY5POR7l4RZkJTMttg&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\ef093ed2da.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcTARKBxq012nuhRedAAsd-bxtM0H4jI5CSSnw&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\1f53b471ba.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcRdpB2v_DZDl9XUJkcA7EQRfXcSQl34u2fPwQ&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\e3d3ade7d9.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcT_ZbLRA-qGEyy9H_Ch1-Nobcw9ZaSqXFoj1g&usqp=CAU - as C:\Users\vivek\Desktop\New folder\images\terrorist\indian_terrorist_photos\5ff9f4264d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQz61wx841EkDzIOzLxZi19DDlwI9EOnOJUlg&usqp=CAU - as C:\Users\vivek\Desktop\Ne